In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI_GCN
import scvi
import pandas as pd
from sklearn.model_selection import train_test_split

import random 
random.seed(2023)


[rank: 0] Global seed set to 0


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [4]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    import random 
    random.seed(2023)
    overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    overlap_gene = sorted(overlap_gene)
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [5]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

In [6]:
info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

In [7]:
seq_data = seq_data[:,info_gene]
spatial_data = spatial_data[:,info_gene]

In [8]:
# scanpy.pp.filter_cells(spatial_data, min_counts=1)
# scanpy.pp.filter_cells(seq_data, min_counts=1)

In [9]:
seq_data.obs['names'] = seq_data.obs_names
spatial_data.obs['names'] = spatial_data.obs_names

seq_data.obs['ind_x'] = seq_data.obs_names
spatial_data.obs['ind_x'] = spatial_data.obs_names

spatial_index = [list(spatial_data.obs['x_coord']), list(spatial_data.obs['y_coord'])]
spatial_data.obsm['spatial'] = np.array(spatial_index).T.astype('int')

/tmp/ipykernel_3934186/2654861310.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  seq_data.obs['names'] = seq_data.obs_names
/tmp/ipykernel_3934186/2654861310.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  spatial_data.obs['names'] = spatial_data.obs_names


In [10]:
import random 
random.seed(2023)
g1 = list(set(spatial_data.var_names).intersection(seq_data.var_names))
g1 = sorted(g1)
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
spatial_data_partial = spatial_data[:, train_g].copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial

# remove cells with no counts
# scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
# scanpy.pp.filter_cells(seq_data, min_counts=1)

# setup_anndata for spatial and sequencing data
# GIMVI_GCN.setup_anndata(spatial_data_partial, labels_key="scClassify", batch_key="batch")
GIMVI_GCN.setup_anndata(spatial_data_partial, labels_key="scClassify")
GIMVI_GCN.setup_anndata(seq_data, labels_key="scClassify")
# GIMVI.setup_anndata(seq_data, labels_key="graph_cluster_anno")

# spatial_data should use the same cells as our training data
# cells may have been removed by scanpy.pp.filter_cells()
spatial_data = spatial_data[spatial_data_partial.obs_names]

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
for i in range(0,10):
    scvi.settings.seed = i
    # create our model

    # setup_anndata for spatial and sequencing data
    GIMVI_GCN.setup_anndata(spatial_data_partial, batch_key="batch", obs_names = 'names')
    GIMVI_GCN.setup_anndata(seq_data)
    model = GIMVI_GCN(seq_data, spatial_data_partial,n_latent = 32)

    # train for 200 epochs
    model.train(300)

    fish_imputation_norm = model.get_imputed_values(normalized=True)[0]
    fish_imputation_raw = model.get_imputed_values(normalized=False)[0]
    fish_imputation_theta = model.get_imputed_theta(normalized=False)[0]

    spatial_data_imputed = spatial_data[:, seq_data.var_names]

    spatial_data_imputed.obsm['imputed'] = fish_imputation_norm
    spatial_data_imputed.obsm['imputed_raw'] = fish_imputation_raw
    spatial_data_imputed.obsm['imputed_raw_theta'] =  fish_imputation_theta
    
#     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
    spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_32_300{i}.h5ad")


[rank: 0] Global seed set to 0
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU availabl

Epoch 300/300: 100%|██████████| 300/300 [04:38<00:00,  1.08it/s, loss=15.4, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [04:38<00:00,  1.08it/s, loss=15.4, v_num=1]


[rank: 0] Global seed set to 1
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU availabl

...

Epoch 300/300: 100%|██████████| 300/300 [04:37<00:00,  1.08it/s, loss=15.6, v_num=1]


[rank: 0] Global seed set to 9
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/data/_utils.py:172: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/scvi/model/_utils.py:286: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/scgpt/l ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU availabl

Epoch 300/300: 100%|██████████| 300/300 [04:36<00:00,  1.09it/s, loss=15.6, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 300/300: 100%|██████████| 300/300 [04:36<00:00,  1.08it/s, loss=15.6, v_num=1]
